### Notebook to be used to test that ATCamera, ATHeaderService and ATArchiver are online and working

In [ ]:
import numpy as np
from lsst.ts import salobj
import asyncio
from astropy.io import fits

import warnings
#import matplotlib.pyplot as plt  # imported as py above
from astropy.modeling import models, fitting
from scipy.ndimage.filters import gaussian_filter
from matplotlib import pyplot as plt
from astropy import time as astropytime
%matplotlib ipympl
mag=4
plt.rcParams['figure.figsize'] = [7*mag, 6*mag]

import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')
import time
import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.geom

import os
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
logger = logging.getLogger('image_display_notebook')

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
# Want to display in firefly?
afwDisplay.setDefaultBackend('firefly')
# os.environ['FIREFLY_HTML'] = "slate.html"
# os.environ['FIREFLY_URL'] = 'http://139.229.170.210:8080/firefly/'
print(os.environ['FIREFLY_HTML'])
print(os.environ['FIREFLY_URL'])

In [ ]:
# # Setup the butler
#accs_images = True
accs_images = False
if accs_images:
    repo = os.path.join("/home/saluser/ingest/accs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    butler = dafPersist.Butler(repo)
else:
    #repo = os.path.join("/home/saluser/ingest/dmcs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    repo = os.path.join("/project/shared/auxTel/")
    #repo = os.path.join("/readonly/lsstdata/auxtel/base/auxtel/oods/butler/repo/")
    butler = dafPersist.Butler(repo) #
    #butler = dafPersist.Butler(repo, mapper="lsst.obs.lsst.auxTel.AuxTelMapper")
#test   
#raw = butler.get("raw", visit=2019111300004)
#image = raw.getImage().array

In [ ]:
visitID = 2021012100658  #132 #292
#visitID = 20210115000015

In [ ]:
# from lsst.rapid.analysis.bestEffort import BestEffortIsr
# REPODIR = repo
# bestEffort = BestEffortIsr(REPODIR)
# isr_corr_exposure = bestEffort.getExposure(visitID)

In [ ]:
data_ref = butler.dataRef("raw", **dict(expId=visitID))
#tmp=data_ref.get()
#image = raw.getImage().array

In [ ]:
from lsst.ip.isr.isrTask import IsrTask
isr_config = IsrTask.ConfigClass()
isr_config.doLinearize = False
isr_config.doBias = True
isr_config.doFlat = False
isr_config.doDark = False
isr_config.doFringe = False
isr_config.doDefect = True
#isr_config.doAddDistortionModel = False
isr_config.doSaturationInterpolation = False
isr_config.doSaturation = False
isr_config.doWrite = False

In [ ]:
isrTask = IsrTask(config=isr_config)

In [ ]:
file = isrTask.runDataRef(data_ref).exposure
isr_corr_exposure = file

In [ ]:
tmp=file.getMetadata()
print(tmp['OBJECT'])

In [ ]:
plt.close('all')
disp = afwDisplay.Display(2, reopenPlot=True, verbose=True)

In [ ]:
if True: # display the image in firefly
    #plt.close('all')
    #disp = afwDisplay.Display(2, reopenPlot=True)
    disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
    disp.setImageColormap('gray')
    disp.scale('linear', 'zscale')
    disp.mtv(isr_corr_exposure, title='visit = {}'.format(visitID))
    #cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)

In [ ]:
# pixels = tmp.image.array
# #pixels=isr_corr_exposure.image.array
# med=np.median(pixels)
# sigma=np.std(pixels)
# dsig = 1
# print(f'Median is {med}, stddev is {sigma}')

In [ ]:
# plt.imshow(isr_corr_exposure.image.array,vmin=med-dsig*sigma,vmax=med+dsig*sigma, origin='lower')

In [ ]:
# Only here for me to debug, should be up top with other declarations
import importlib
import utils.findNarrowbandRonchiPeaks
importlib.reload(utils.findNarrowbandRonchiPeaks)
from utils.findNarrowbandRonchiPeaks import findNarrowbandRonchiPeaks

import utils.fitExposure
importlib.reload(utils.fitExposure)
from utils.fitExposure import fit_2d_PSF

import utils.calc_CofM
importlib.reload(utils.calc_CofM)
from utils.calc_CofM import calc_CofM

import utils.calc_encircled_energy
importlib.reload(utils.calc_encircled_energy)
from utils.calc_encircled_energy import calc_encircled_energy

import utils.findBrightestSourceInBox
importlib.reload(utils.findBrightestSourceInBox)
from utils.findBrightestSourceInBox import findBrightestSourceInBox

In [ ]:
isr_corr_exposure.image.array -= np.median(isr_corr_exposure.image.array)

In [ ]:
# Source detection libraries
from lsst.meas.algorithms.detection import SourceDetectionTask
import lsst.afw.table as afwTable

# create the output table for source detection
schema = afwTable.SourceTable.makeMinimalSchema()
config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 20  # detection threshold after smoothing
# config.nSigmaToGrow = 1.2 # Only for doughnuts

sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)
tab = afwTable.SourceTable.make(schema)
result = sourceDetectionTask.run(tab, isr_corr_exposure, sigma=5.1)

In [ ]:
#result.sources

## Fit a gaussian

In [ ]:
# Find 0th and +/- 1 order peaks 
sources = result.sources
zeroth_order_estimate = lsst.geom.Point2D(2180, 2818)
#zeroth_order_estimate = lsst.geom.Point2D(2250,1500)
zeroth_order_star_BBox= lsst.geom.Box2I.makeCenteredBox(zeroth_order_estimate, lsst.geom.Extent2I(400,400)) 
bbox0 = lsst.geom.Box2I.makeCenteredBox(zeroth_order_estimate, lsst.geom.Extent2I(100,100)) 
wavelength= 677.0 
dispersion =  0 #(1/0.6358) # pixels/nm
spectral_position_angle= 0.0107 # radians clockwise from top
#center_source, peak1, peak2 = findNarrowbandRonchiPeaks(sources, zeroth_order_star_BBox, wavelength, dispersion, spectral_position_angle)

center_source, center_peak = findBrightestSourceInBox(sources, zeroth_order_star_BBox)

peak0_subim = isr_corr_exposure.subset(bbox0)
model='moffat2d'
#model='Gaussian2d'

p0, x0 , y0 = fit_2d_PSF(peak0_subim, plot=True, model=model)
print('Visit ID = {}'.format(visitID))

In [ ]:
plate_scale = 0.101
if model.lower() == 'gaussian2d':
    print(f'x_FWHM = {p0.x_fwhm*plate_scale:0.3f} arcsec')
    print(f'y_FWHM = {p0.y_fwhm*plate_scale:0.3f} arcsec')
if model.lower() == 'moffat2d':
    print(f'Moffat FWHM = {p0.fwhm*plate_scale:0.3f} arcsec')
    print(f'Moffat FWHM = {p0.fwhm*plate_scale:0.3f} arcsec')

In [ ]:
# peak0_subim = isr_corr_exposure.subset(bbox0)
# p0, x0 , y0 = fit2DGaussian(peak0_subim, plot=True)

In [ ]:

# # Fit peaks
# # zeroth order
# # variables names are weird here because I can't think of a clever way to have -1 and +1 as variable names

# #bbox0 = lsst.geom.Box2I.makeCenteredBox(center_source.getFootprint().getCentroid(), lsst.geom.Extent2I(100,100)) 

# print('visitID is: {}'.format(visitID))
# zeroth_order_estimate = lsst.geom.Point2D(1630,1330)
# #zeroth_order_estimate = lsst.geom.Point2D(2025,1980)
# bbox0 = lsst.geom.Box2I.makeCenteredBox(zeroth_order_estimate, lsst.geom.Extent2I(100,100)) 

# peak0_subim = isr_corr_exposure.subset(bbox0)
# p0, x0 , y0 = fit2DGaussian(peak0_subim, plot=True)


In [ ]:
p0_x_CofM, p0_y_CofM = calc_CofM(peak0_subim) # 2167,3372

# Calculate EE and CofM
p0_EE_rad50_pix, p0_EE_rad67_pix, p0_EE_rad80_pix = calc_encircled_energy(peak0_subim, plot=True) 
fit_data.append(p0_EE_rad80_pix)